In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import logging
import json
import subprocess as sp
import numpy as np

from PIL import Image

from astropy.io import fits

from sdss_gz_data import scale_rgb

In [3]:
PROJECT = 'astroml-ast80014'
SUBSCRIPTION_NAME = 'cutout-fits-image'
SUBSCRIPTION = f'projects/{PROJECT}/subscriptions/{SUBSCRIPTION_NAME}'
BUCKET = 'wgauvin-astroml-ast80014'
DATA_DIR = '/tmp'
SIZING_RATIO = 7.8567420132
CUTOUT_RATIO = 2 * SIZING_RATIO
CUTOUT_PIXEL_SIZE = 72
JPEG_SIZE = 144

TOP_COORD = int(CUTOUT_PIXEL_SIZE / 4)
BTTM_COORD = int(TOP_COORD + CUTOUT_PIXEL_SIZE / 2)

In [4]:
def cleanup(msg):
    import shutil

    shutil.rmtree(msg['workdir'], ignore_errors=True)

In [5]:
def save_fits(data, wcs, filename):
    header = wcs.to_header()
    hdu = fits.PrimaryHDU(data, header=header)
    hdu.writeto(filename, overwrite=True)

def save_cutout(cutout, filename):
    save_fits(cutout.data, cutout.wcs, filename)

In [6]:
def cutout_img_bands(msg):
    from astropy.coordinates import SkyCoord, ICRS
    import astropy.units as u
    from astropy.wcs import WCS
    from astropy.nddata import Cutout2D

    run = msg['run']
    camcol = msg['camcol']
    field = msg['field']
    objid = msg['objid']
    ra = msg['ra']
    dec = msg['dec']
    petroRad_r = msg['petroRad_r']
    workdir = msg['workdir']

    angular_size = CUTOUT_RATIO * petroRad_r

    bands = ['i', 'r', 'g']
    cutout_filenames = np.empty(3, dtype=object)

    position = SkyCoord(ra=ra, dec=dec, frame=ICRS, unit=u.deg)
    cutout_size = u.Quantity((angular_size, angular_size), u.arcsec)

    for idx, filename in enumerate(msg['filenames']):
        band = bands[idx]
        fits_file = fits.open(filename)
        hdu = fits_file[0]
        data = hdu.data
        header = hdu.header
        wcs = WCS(header, naxis=2)
    
        cutout = Cutout2D(data, position=position, size=cutout_size, wcs=wcs)

        cutout_filename = f'{workdir}/obj-{objid}-{band}.fits.bz2'
        save_cutout(cutout, cutout_filename)
        cutout_filenames[idx] = cutout_filename

    msg['cutout_filenames'] = cutout_filenames
    
    return msg

In [7]:
# DONT COPY BACK
def get_object_msgs(msg):
    msgs = np.empty_like(msg['objects'])
    
    for idx, obj in enumerate(msg['objects']):
        msgs[idx] = {
            'run': msg['run'],
            'camcol': msg['camcol'],
            'field': msg['field'],
            'objid': obj['objid'],
            'ra': obj['ra'],
            'dec': obj['dec'],
            'petroRad_r': obj['petroRad_r'],
            'filenames': msg['filenames'],
            'workdir': msg['workdir']
        }

    return msgs

In [8]:
def save_jpeg(msg, data, vmax):
    
    data = data.copy()

    data = data[:, TOP_COORD:BTTM_COORD, TOP_COORD:BTTM_COORD]
    jpeg_data = scale_rgb(data)

    image = Image.fromarray(np.transpose(jpeg_data))
    image = image.resize((JPEG_SIZE, JPEG_SIZE), Image.BICUBIC)
    image = image.transpose(Image.ROTATE_90)
    image.save(filename)

    msg['jpeg_file'] = filename

In [9]:
def create_jpeg(msg):
    objid = msg['objid']
    workdir = msg['workdir']

    filename = f'{workdir}/obj-{objid}.jpeg'
    
    fits_file = fits.open(msg['fits_image'])
    data = fits_file[0].data
    
    data = data[:, TOP_COORD:BTTM_COORD, TOP_COORD:BTTM_COORD]
    jpeg_data = scale_rgb(data)

    image = Image.fromarray(np.transpose(jpeg_data))
    image = image.resize((JPEG_SIZE, JPEG_SIZE), Image.BICUBIC)
    image = image.transpose(Image.ROTATE_90)
    image.save(filename)

    msg['jpeg_file'] = filename

In [24]:
# CHANGED - migrate
def create_data_cube(msg):
    from aplpy.rgb import make_rgb_cube

    workdir = msg['workdir']
    objid = msg['objid']

    cutout_filenames = np.empty(3, dtype=object)
    
    for idx, band in enumerate(['i', 'r', 'g']):
        cutout_filenames[idx] = get_obj_filename(workdir, objid, band)
    
    outfile = get_obj_filename(workdir, objid)
    make_rgb_cube(cutout_filenames, outfile)

    msg['fits_image'] = outfile
    return msg

In [21]:
def create_workdir(msg):
    import os

    run = msg['run']
    camcol = msg['camcol']
    field = msg['field']

    workdir = f'{DATA_DIR}/{run}-{camcol}-{field}'

    try:
        os.makedirs(workdir, exist_ok=True)
    finally:
        msg['workdir'] = workdir

    return msg

In [12]:
# def retrieve_fits_files(msg):
#     from shutil import copy

#     filenames = np.empty(3, dtype=object)

#     get_file_from_gcs(msg, 0, 'i', filenames),
#     get_file_from_gcs(msg, 1, 'r', filenames),
#     get_file_from_gcs(msg, 2, 'g', filenames)

#     msg['filenames'] = filenames
#     return msg

In [13]:
def get_file_from_gcs(msg, idx, band, filenames):
    run = msg['run']
    camcol = msg['camcol']
    field = msg['field']
    workdir = msg['workdir']

    file_dir = f'fits/{run}/{camcol}/{field}'
    filename = f'frame-{band}-{run:06d}-{camcol}-{field:04d}.fits.bz2'

    infile = f'{file_dir}/{filename}'
    outfile = f'{workdir}/{filename}'

    print(f'Downloading {infile}')
    cmd = f'gsutil cp gs://{BUCKET}/{infile} {outfile} > /dev/null 2>&1'
    sp.run(cmd, shell=True, text=True, check=True)
    filenames[idx] = outfile

In [15]:
def fill_nan(data):
    nans = np.isnan(data)
    data = np.nan_to_num(data)
    data[nans] = -100
    
    return data

def resize(data, size):
    return np.array(Image.fromarray(data).resize((size, size), Image.BICUBIC))

def cutout_fits(msg):
    from astropy.coordinates import SkyCoord, ICRS
    import astropy.units as u
    from astropy.wcs import WCS
    from astropy.wcs.utils import skycoord_to_pixel
    from astropy.nddata import Cutout2D

    run = msg['run']
    camcol = msg['camcol']
    field = msg['field']
    objid = msg['objid']
    ra = msg['ra']
    dec = msg['dec']
    petroRad_r = msg['petroRad_r']
    workdir = msg['workdir']
    fits_image = msg['fits_image']

    print(f'Creating cutout for - run: {run}, camcol: {camcol}, field: {field}, objid: {objid}, ra: {ra}, dec: {dec}, petroRad_r: {petroRad_r}')
    
    fits_file = fits.open(fits_image)
    hdu = fits_file[0]
    data = hdu.data
    wcs = WCS(hdu.header, naxis=2)
    
    angular_size = SIZING_RATIO * petroRad_r

    position = SkyCoord(ra=ra, dec=dec, frame=ICRS, unit=u.deg)
    cutout_size = u.Quantity((angular_size, angular_size), u.arcsec)

    out_data = np.empty((3, CUTOUT_PIXEL_SIZE, CUTOUT_PIXEL_SIZE))
    cutout_wcs = WCS(naxis=3)

    for idx in range(3):
        cutout = Cutout2D(data[idx], position=position, size=cutout_size, wcs=wcs)
        if (idx == 0):
            curr_wcs = cutout.wcs.wcs
            cutout_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN', '']
            cutout_wcs.wcs.cdelt = np.array([curr_wcs.cdelt[0], curr_wcs.cdelt[1], 1.0])
            cutout_wcs.wcs.crpix = [curr_wcs.crpix[0], curr_wcs.crpix[1], 0.0]
            cutout_wcs.wcs.crval = [curr_wcs.crval[0], curr_wcs.crval[1], 0.0]
        
        temp_data = fill_nan(cutout.data)
        out_data[idx] = resize(temp_data, CUTOUT_PIXEL_SIZE)

    save_fits(out_data, cutout_wcs, fits_image)
    
    return msg

In [16]:
msg = {
    'run': 4678,
    'camcol': 3,
    'field': 162,
    'objects': [
        {
            'objid': '1237665566074994868',
            'ra': 225.965278271111,
            'dec': 19.2838103331967,
            'petroRad_r': 5.737516
        },
        {
            'objid': '1237665566074994872',
            'ra': 225.938960804469,
            'dec': 19.1965682139844,
            'petroRad_r': 7.560683
        }
    ]
}

In [26]:
def get_obj_filename(workdir, objid, band=None):
    if (band is None):
        return f'{workdir}/obj-{objid}.fits.bz2'
    else:
        return f'{workdir}/obj-{objid}-{band}.fits.bz2'

def cutout_img_band(msg, obj_msg, idx, band, filename):
    from astropy.coordinates import SkyCoord, ICRS
    import astropy.units as u
    from astropy.wcs import WCS
    from astropy.nddata import Cutout2D

    run = msg['run']
    camcol = msg['camcol']
    field = msg['field']
    workdir = msg['workdir']

    objid = obj_msg['objid']
    ra = obj_msg['ra']
    dec = obj_msg['dec']
    petroRad_r = obj_msg['petroRad_r']

    angular_size = CUTOUT_RATIO * petroRad_r

    position = SkyCoord(ra=ra, dec=dec, frame=ICRS, unit=u.deg)
    cutout_size = u.Quantity((angular_size, angular_size), u.arcsec)

    fits_file = fits.open(filename)
    hdu = fits_file[0]
    data = hdu.data
    header = hdu.header
    wcs = WCS(header, naxis=2)

    cutout = Cutout2D(data, position=position, size=cutout_size, wcs=wcs)

    cutout_filename = get_obj_filename(workdir, objid, band)
    save_cutout(cutout, cutout_filename)

    return msg

In [31]:
def get_object_msgs(msg):
    for obj in msg['objects']:
        obj_msg = {
            'run': msg['run'],
            'camcol': msg['camcol'],
            'field': msg['field'],
            'objid': obj['objid'],
            'ra': obj['ra'],
            'dec': obj['dec'],
            'petroRad_r': obj['petroRad_r'],
            'filenames': msg['filenames'],
            'workdir': msg['workdir']
        }
        yield obj_msg

In [35]:
def process_band(msg, idx, band):
    filenames = msg['filenames']
    get_file_from_gcs(msg, idx, band, filenames)
    
    for obj_msg in msg['objects']:
        cutout_img_band(msg, obj_msg, idx, band, filenames[idx])

In [36]:
# DONT COPY BACK
msg = create_workdir(msg)
filenames = np.empty(3, dtype=object)
msg['filenames'] = filenames

for idx, band in enumerate(['i', 'r', 'g']):
    process_band(msg, idx, band)

# for idx, band in enumerate(['i', 'r', 'g']):
#     get_file_from_gcs(msg, idx, band, filenames)
    
#     for obj_msg in msg['objects']:
#         cutout_img_band(msg, obj_msg, idx, band, filenames[idx])

# msg = retrieve_fits_files(msg)

In [37]:
for obj_msg in get_object_msgs(msg):
    obj_msg = create_data_cube(obj_msg)
    obj_msg = cutout_fits(obj_msg)
    obj_msg = create_jpeg(obj_msg)

Creating cutout for - run: 4678, camcol: 3, field: 162, objid: 1237665566074994868, ra: 225.965278271111, dec: 19.2838103331967, petroRad_r: 5.737516


/Users/will/Development/Astronomy/proposal/ENV/lib/python3.7/site-packages/sdss_gz_data/sdss_gz_data.py:391: RuntimeWarning: invalid value encountered in true_divide
  scale = slope * arcsinh((mean - min) / sigma) / mean


Creating cutout for - run: 4678, camcol: 3, field: 162, objid: 1237665566074994872, ra: 225.938960804469, dec: 19.1965682139844, petroRad_r: 7.560683


In [ ]:
msg

In [ ]:
obj_msg = {
    'run': 4678,
    'camcol': 3,
    'field': 162,
    'objid': '1237665566074994868',
    'ra': 225.965278271111,
    'dec': 19.2838103331967,
    'petroRad_r': 5.737516,
    'filenames': np.array([
        '/tmp/4678-3-162/frame-i-004678-3-0162.fits.bz2',
        '/tmp/4678-3-162/frame-r-004678-3-0162.fits.bz2',
        '/tmp/4678-3-162/frame-g-004678-3-0162.fits.bz2'
    ]),
    'workdir': '/tmp/4678-3-162'
}

In [ ]:
obj_msg = cutout_img_bands(obj_msg)

In [ ]:
obj_msg

In [ ]:
i_fits = fits.open(obj_msg['cutout_filenames'][0])

In [ ]:
i_fits_data = i_fits[0].data
print(i_fits_data)
print(np.min(i_fits_data), np.max(i_fits_data))

In [ ]:
from astropy.wcs import WCS

i_fits_wcs = WCS(i_fits[0].header)
i_fits_wcs

In [ ]:
r_fits = fits.open(obj_msg['cutout_filenames'][1])
r_fits_data = r_fits[0].data
print(r_fits_data)
print(np.min(r_fits_data), np.max(r_fits_data))
r_fits_wcs = WCS(r_fits[0].header)
r_fits_wcs

In [ ]:
g_fits = fits.open(obj_msg['cutout_filenames'][2])
g_fits_data = g_fits[0].data
print(i_fits_data)
print(np.min(i_fits_data), np.max(i_fits_data))
g_fits_wcs = WCS(g_fits[0].header)
g_fits_wcs

In [ ]:
obj_msg = create_data_cube(obj_msg)
obj_msg

In [ ]:
datacube_fits = fits.open(obj_msg['fits_image'])
datacube_fits_data = datacube_fits[0].data
datacube_fits_wcs = WCS(datacube_fits[0].header)
datacube_fits_wcs

In [ ]:
datacube_fits_data[0,30:-30,30:-30].shape

In [ ]:
np.max(datacube_fits_data)

In [ ]:
nans = np.isnan(datacube_fits_data)
datacube_fits_data = np.nan_to_num(datacube_fits_data)
datacube_fits_data[nans] = -100

In [ ]:
datacube_fits_data